# Kunskapskontroll 2

The aim of this project is to use Python code to scrape, clean, structure and store data automatically. The website containing the data needed is "E-reklamblad.se" which updates with new product offers in store every week. The idea is to build a scraper that will fetch the weekly products on offer, run this through a data cleaning pipeline and finally store it in a SQL database. The idea is to further be able to connect the database with a LLM model in order to search for products on offer and have the LLM generate recipe ideas from the products on offer combined with what is currently in the fridge (user input).

In [1]:
import pandas as pd
from scraper import Scraper
from loader import DataLoader
from cleaner import DataCleaner

In [77]:
extracted_data = []

# Iterate through each product in the list
for item in all_offers:
    if isinstance(item, dict):  # Ensure item is a dictionary
        product = {
            'key_name': item.get('heading', 'N/A'),  # Use 'N/A' if 'heading' key is missing
            'price': item.get('pricing', {}).get('price', 'N/A'),  # Adjust based on actual structure if 'pricing' key does not exist
            'description': item.get('description', 'N/A')  # Use 'N/A' if 'description' key is missing
        }
        extracted_data.append(product)
    else:
        print(f"Unexpected item format: {item}")

# Print extracted data
for product in extracted_data:
    print(f"Key Name: {product['key_name']}, Price: {product['price']}, Details: {product['description']}")

Unexpected item format: [{'id': 'vswJwg_pMHppnHn8qjCQI', 'ern': 'ern:offer:vswJwg_pMHppnHn8qjCQI', 'heading': 'Ekologisk nötfärs', 'description': 'Obs! Medlemspris | Garant Eko, Sverige, färsk, fetthalt max 12%, 500g, jfr-pris 119,90/kg.', 'catalog_page': 5, 'catalog_view_id': None, 'pricing': {'price': 59.95, 'pre_price': None, 'currency': 'SEK'}, 'quantity': {'unit': {'symbol': 'g', 'si': {'symbol': 'kg', 'factor': 0.001}}, 'size': {'from': 500, 'to': 500}, 'pieces': {'from': 1, 'to': 1}}, 'images': {'thumb': 'https://image-transformer-api.tjek.com/?u=s3%3A%2F%2Fsgn-prd-assets%2Fuploads%2FMA1cYEVg%2Fp-5.webp&w=300&x1r=0.0227&x2r=0.633&y1r=0.2758&y2r=0.4991&s=aa7e18edf8bb17e1ff7918709d90d59d', 'view': 'https://image-transformer-api.tjek.com/?u=s3%3A%2F%2Fsgn-prd-assets%2Fuploads%2FMA1cYEVg%2Fp-5.webp&w=600&x1r=0.0227&x2r=0.633&y1r=0.2758&y2r=0.4991&s=9f450a9e87dbd87236562984a8deae8b', 'zoom': 'https://image-transformer-api.tjek.com/?u=s3%3A%2F%2Fsgn-prd-assets%2Fuploads%2FMA1cYEVg%2Fp

In [71]:
# Example catalog ID from your response
catalog_id = 'MA1cYEVg'

# URL to fetch offers (replace with the correct URL if available)
offers_url = f'https://ereklamblad.se/api/squid/v2/offers?catalog_id={catalog_id}'

response = requests.get(offers_url)
response.raise_for_status()

print(response.headers)

# Assuming response is a requests.Response object
#soup = BeautifulSoup(response.content)
#print(soup)

import json
data = json.loads(response.content)
print(data)
# List to hold extracted data
extracted_data = []

# Iterate through each product in the list
for item in data:
    product = {
        'key_name': item.get('heading', 'N/A'),  # Use 'N/A' if 'heading' key is missing
        'price': item.get('pricing', {}).get('price', 'N/A'),       # Use 'N/A' if 'price' key is missing
        'description': item.get('description', 'N/A')    # Use 'N/A' if 'details' key is missing
    }
    extracted_data.append(product)

# Print extracted data
for product in extracted_data:
    print(f"Key Name: {product['key_name']}, Price: {product['price']}, Details: {product['description']}")

{'vary': 'Origin, Accept-Encoding', 'access-control-allow-credentials': 'true', 'access-control-expose-headers': 'X-Token,X-Token-Expires,X-Api-Key,X-Client-Version', 'referrer-policy': 'origin-when-cross-origin, strict-origin-when-cross-origin', 'x-content-type-options': 'nosniff', 'content-type': 'application/json; charset=utf-8', 'cache-control': 'private', 'alt-svc': 'h3=":443"; ma=2592000,h3-29=":443"; ma=2592000, h3=":443"; ma=2592000,h3-29=":443"; ma=2592000', 'access-control-allow-origin': '*', 'set-cookie': 'sgn-session=; path=/; httpOnly; expires=Thu, 05 Sep 2024 18:44:11 GMT; sameSite=lax; secure; domain=ereklamblad.se', 'content-encoding': 'gzip', 'date': 'Fri, 06 Sep 2024 18:44:11 GMT', 'server': 'Google Frontend', 'via': '1.1 google', 'Transfer-Encoding': 'chunked'}
[{'id': 'vswJwg_pMHppnHn8qjCQI', 'ern': 'ern:offer:vswJwg_pMHppnHn8qjCQI', 'heading': 'Ekologisk nötfärs', 'description': 'Obs! Medlemspris | Garant Eko, Sverige, färsk, fetthalt max 12%, 500g, jfr-pris 119,90

In [67]:
import json 

api_url = 'https://ereklamblad.se/api/squid/v4/rpc/get_offers'
response = requests.get(api_url)
response.raise_for_status()

# Assuming response is a requests.Response object
data = response.json()
print(data)

# Extract all city values
#cities = {store.get('city') for store in data if store.get('city')}
#print(cities)

# Filter for stores in Malmö and extract their IDs
#malmo_ids = [store['id'] for store in data if store['city'] == 'Malmö']

#print(malmo_ids)

{'name': 'get_offers', 'description': 'Fetch a paginated list of Offers, filtered by the provided search criteria.', 'input': {'type': 'object', 'properties': {'page': {'type': 'object', 'properties': {'after_cursor': {'type': 'string'}, 'page_size': {'type': 'integer', 'minimum': 1, 'maximum': 24, 'default': 24}}, 'additionalProperties': False, 'description': 'The input you use to define the start of a paginated request, and the max number of items in the result.', 'default': {'page_size': 24}}, 'where': {'type': 'object', 'properties': {'term': {'type': 'string', 'maxLength': 40, 'description': 'The query string to search for.'}, 'max_radius': {'type': 'integer', 'minimum': 1, 'description': 'The distance around the geohash, in meters, to search.'}, 'business_ids': {'type': 'array', 'items': {'type': 'string'}, 'minItems': 1, 'description': 'The Ids of the Businesses to limit the search to.'}, 'business_category_ids': {'type': 'array', 'items': {'type': 'string'}, 'minItems': 1, 'des

In [31]:

malmo_ids = [store['business_id'] for store in data if store['city'] == 'Malmö']
print(malmo_ids)

[]


In [29]:
api_url = 'https://ereklamblad.se/api/squid/v4/rpc/get_offers'
response = requests.get(api_url)
response.raise_for_status()
data = response.json()
data

{'name': 'get_offers',
 'description': 'Fetch a paginated list of Offers, filtered by the provided search criteria.',
 'input': {'type': 'object',
  'properties': {'page': {'type': 'object',
    'properties': {'after_cursor': {'type': 'string'},
     'page_size': {'type': 'integer',
      'minimum': 1,
      'maximum': 24,
      'default': 24}},
    'additionalProperties': False,
    'description': 'The input you use to define the start of a paginated request, and the max number of items in the result.',
    'default': {'page_size': 24}},
   'where': {'type': 'object',
    'properties': {'term': {'type': 'string',
      'maxLength': 40,
      'description': 'The query string to search for.'},
     'max_radius': {'type': 'integer',
      'minimum': 1,
      'description': 'The distance around the geohash, in meters, to search.'},
     'business_ids': {'type': 'array',
      'items': {'type': 'string'},
      'minItems': 1,
      'description': 'The Ids of the Businesses to limit the sea

In [25]:
response = requests.get('https://squid-api.tjek.com/v2/catalogs/MA1cYEVg/offers')
response.raise_for_status()
offers = response.json()
print(offers)

HTTPError: 404 Client Error: Not Found for url: https://squid-api.tjek.com/v2/catalogs/MA1cYEVg/offers

In [ ]:
url = "https://www.ereklamblad.se/ICA-Supermarket/erbjudanden"
scraper = Scraper(url)
scraper.scrape()

In [2]:
folder_path = '.'
data = DataLoader(folder_path)
df = data.load_data_to_df()

Loading file: .\offers_2024-09-06.csv


In [3]:
df.head()

,Name,Price,Details
0,Riven ost,32 kr,"2x150 g•106,67 kr/kg"
1,Vitost,35 kr,"2x150 g•116,67 kr/kg"
2,Vällagrad ost,"119,90 kr","1 kg•119,90 kr/kg"
3,"Kokosmjölk, Kryddmix Asian/Indian",55 kr,"5x30-40 g•max 366,67 kr/kg"
4,"Schampo, Balsam",50 kr,2x200-250 ml•max 125 kr/l


In [4]:
clean_data = DataCleaner(df)
cleaned_df = clean_data.clean()
cleaned_df.head()


,Name,Price,Quantity,ComparisonPrice
0,Riven ost,32.0,2x150 g,"106,67 kr/kg"
1,Vitost,35.0,2x150 g,"116,67 kr/kg"
2,Vällagrad ost,119.9,1 kg,"119,90 kr/kg"
3,"Kokosmjölk, Kryddmix Asian/Indian",55.0,5x30-40 g,"max 366,67 kr/kg"
4,"Schampo, Balsam",50.0,2x200-250 ml,max 125 kr/l


In [5]:
cleaned_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 85 entries, 0 to 84
Data columns (total 4 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   Name             85 non-null     object 
 1   Price            85 non-null     float64
 2   Quantity         85 non-null     object 
 3   ComparisonPrice  79 non-null     object 
dtypes: float64(1), object(3)
memory usage: 2.8+ KB


In [6]:
cleaned_df.describe()

,Price
count,85.000000
mean,38.955294
std,23.486726
min,10.000000
25%,22.900000
50%,30.000000
75%,49.000000
max,119.900000
